In [1]:
# -*- coding: utf-8 -*-

import tkinter as tk
from tkinter.font import Font
from PIL import Image, ImageTk
from tkinter.ttk import*
from tkinter import filedialog
import cv2
import numpy as np
import dlib
from math import hypot
import math

# 讀取人臉辨識模型
detector = dlib.get_frontal_face_detector()

# 讀取人臉辨識之特徵模型
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

## openimg_file
#### 一、打開貼圖圖片來更換GUI影像
1. 使用filedialog.askopenfilename來跳出檔案窗格選擇，可以調整預顯示的副檔名
2. 使用PIL中的Image來開啟影像，並更改大小為(320,250)
3. 使用ImageTk中的PhotoImage將影像轉為Tk格式
4. 使用config與image將原來的label_img更換影像

#### 二、圖片的格式轉為黑底的貼圖格式
1. 使用cv2中的split來分割三通道(R,G,B)
2. 使用numpy中的empty建立空白的陣列來存放將要處理的影像，並將此陣列存取的元素規定為整數(int)
3. 多巢迴圈(高與寬)與條件式來存放像素：
    a. 背景為白色之外的區域之像素存原來貼圖的像素
    b. 背景為白色的區域之像素存黑色
4. 使用cv2中的imwrite來儲存背景為黑的貼圖

In [2]:
def openimg_file():
    
    # 開啟檔案
    filename = filedialog.askopenfilename(title='Select file', filetypes=[("all files","*.*"),("png files","*.png"),("jpeg files","*.jpg")])
    
    # 更換影像
    img = Image.open(filename).resize((250,250))
    imgtk = ImageTk.PhotoImage(img)
    label_img.config(image = imgtk)
    label_img.image = imgtk
    
    # 打開影像
    postimg = cv2.imread(filename)
    
    # 分割RGB通道
    r,g,b = cv2.split(postimg)

    # 建立放置背景為黑的貼圖之空陣列，並指定此陣列只存放整數
    black_img = np.empty(postimg.shape)
    black_img = black_img.astype(int)

    # 跑寬與高的多巢迴圈，設立條件式：將背景為白色之外的區域之像素存原來貼圖的像素;反之，背景為白色的區域之像素存黑色
    for h in range(black_img.shape[0]):
        for w in range(black_img.shape[1]):
            if ((b[h][w]) == 255)&((g[h][w]) == 255)&((r[h][w]) == 255):
                black_img[h][w][0]=0
                black_img[h][w][1]=0
                black_img[h][w][2]=0
            else:
                black_img[h][w][0]=postimg[h][w][0]
                black_img[h][w][1]=postimg[h][w][1]
                black_img[h][w][2]=postimg[h][w][2]

    # 儲存背景為黑的貼圖影像
    cv2.imwrite("black_img.jpg", black_img)

## opencv_frame
#### 點下Start的按鈕就立即觸發函數opencv_frame，將分為開啟OpenCV的攝影機、影片或影像：
一、當選擇為Camera，則啟動攝影機：
1. 使用cv2中的VideoCapture來啟動攝影鏡頭
2. 無限迴圈來獲取當前畫面
        a. 使用cv2中的resize來更改影像大小
        b. 使用cv2中的imshow來顯現畫面
        c. 使用cv2中的waitKey來決定畫面速度與跳出無限迴圈的按鍵
3. 使用cv2中的release與destroyAllWindows來銷毀視窗與畫面

二、當選擇為Image，則開啟選擇檔案視窗，來選擇欲開啟的相片：
1. 使用filedialog.askopenfilename來跳出檔案窗格選擇，可以調整預顯示的副檔名
2. 使用cv2中的imread開啟相片
3. 使用cv2中的namedWindow與imshow來顯現畫面

三、當選擇為Video，則開啟選擇檔案視窗，來選擇欲開啟的影片：
1. 使用filedialog.askopenfilename來跳出檔案窗格選擇，可以調整預顯示的副檔名

In [5]:
def opencv_frame():

    if openvar.get()=="Camera":
        # 使用攝影頭
        VIDEO_IN = cv2.VideoCapture(0)

        while True:
            hasFrame, frame = VIDEO_IN.read()
            frame = cv2.resize(frame, None, fx=0.8, fy=0.8)
            #進行影像辨識或影像貼圖
        
            cv2.imshow("Camera", frame)
    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        VIDEO_IN.release()
        cv2.destroyAllWindows()
        
    if openvar.get()=="Image":
        imgname = filedialog.askopenfilename(title='Select file', filetypes=[("all files","*.*"),("png files","*.png"),("jpeg files","*.jpg")])
        img = cv2.imread(imgname)
        cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
        cv2.imshow('Image', img)
        
    if openvar.get()=="Video":
        videoname = filedialog.askopenfilename(title='Select file', filetypes=[("all files","*.*"),("mp4 files","*.mp4"),("avi files","*.avi")])
        VIDEO_IN = cv2.VideoCapture(videoname)

In [6]:
#建立視窗
app = tk.Tk()

#建立視窗標題
app.title("Face - Image Posting")

#建立視窗背景
#app.configure(background = 'white')

#建立視窗大小
app.geometry('370x480')

#設立字體
title_label_Font = Font(family="Times", size=18, underline=1)
subtitle_label_Font_1 = Font(family="Times", size=14)

#建立標籤與位置
#建立兩個畫面布局
#布局一
frame_title = tk.Frame(app)
frame_title.grid(column=0, row=0, ipadx=3, pady=3)

#布局二
frame_image = tk.Frame(app)
frame_image.grid(column=0, row=1, ipadx=3, pady=3)

#布局三
frame_input = tk.Frame(app)
frame_input.grid(column=0, row=2, ipadx=3, pady=3)

#布局四
frame_output = tk.Frame(app)
frame_output.grid(column=0, row=3, ipadx=3, pady=3)

#放置標題
label_Title = tk.Label(frame_title, text="Face - Image Posting", font=title_label_Font)
label_Title.grid(column=0, row=0, ipadx=100, pady=0)

#放置副標題
label_Subtitle = tk.Label(frame_title, text="※ Please enter the correct image (.png/.jpg).", font=subtitle_label_Font_1)
label_Subtitle.grid(column=0, row=1, ipadx=5, pady=5)

#放置圖片
img_path = "Image_test/pre-inserted picture.png"
img = Image.open(img_path).resize((250,250))
imgtk = ImageTk.PhotoImage(img)
label_img = tk.Label(frame_image, image=imgtk)
label_img.grid(column=0, row=0, ipadx=5, pady=0)

#輸入貼圖的放大倍數
label_scaling = tk.Label(frame_input, text = "Scaling ", font=subtitle_label_Font_1)
label_scaling.grid(column=0, row=0, pady=0)
scaling = tk.Entry(frame_input)
scaling.grid(column=1, row=0, pady=0)

#選擇貼圖放置的位置
label_imgloc = tk.Label(frame_input, text = "Location", font=subtitle_label_Font_1)
label_imgloc.grid(column=0, row=1, pady=0)
imglocvar = tk.StringVar()
imglocCB = Combobox(frame_input, textvariable = imglocvar)
imglocCB["value"]=("Nose","Head","Eye","Mouth")
imglocCB.grid(column=1, row=1, padx=10)

#選擇貼圖為影片,相片或攝影機串流
label_open = tk.Label(frame_input, text = "Open Image", font=subtitle_label_Font_1)
label_open.grid(column=0, row=2, pady=0)
openvar = tk.StringVar()
openCB = Combobox(frame_input, textvariable = openvar)
openCB["value"]=("Image","Video","Camera")
openCB.grid(column=1, row=2, padx=10)

#放置按鈕與位置
imgButton = tk.Button(frame_output, text = 'Open', command = openimg_file)
imgButton.grid(column=0, row=2, pady=10)

openButton = tk.Button(frame_output, text = 'Start', command = opencv_frame)
openButton.grid(column=1, row=2, pady=10)

#程式開始迴圈
app.mainloop()